In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dir = '/content/drive/MyDrive/Courses/2021fall/Spoken Language Technologies/Lab3/data/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
import librosa
import math
import numpy as np
import scipy.signal
from scipy.special import logsumexp
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, (1, 5))
        self.fc1 = nn.Linear(128, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 48)
        self.sm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 128)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sm(x)
        return x
    
def load_audio_to_melspec_tensor(wavpath, sample_rate=16000):
    window_size = .025
    window_stride = 0.01
    n_dft = 512
    win_length = int(sample_rate * window_size)
    hop_length = int(sample_rate * window_stride)
    y, sr = librosa.load(wavpath, sr=sample_rate)
    y = y - y.mean()
    y = np.append(y[0],y[1:]-.97*y[:-1])
    # compute mel spectrogram
    stft = librosa.stft(y, n_fft=n_dft, hop_length=hop_length,
        win_length=win_length, window=scipy.signal.hamming)
    spec = np.abs(stft)**2
    mel_basis = librosa.filters.mel(sample_rate, n_dft, n_mels=40, fmin=20)
    melspec = np.dot(mel_basis, spec)
    logspec = librosa.power_to_db(melspec, ref=np.max)
    logspec = np.transpose(logspec)
    logspec_tensor = torch.tensor(logspec)
    return logspec_tensor

def compute_phone_likelihoods(model, logspec):
    likelihood_list = []
    with torch.no_grad():
        for j in range(6, logspec.size(0) - 5):
            inp = logspec[j-5:j+6,:].unsqueeze(0)
            output = model(inp) # output will be log probabilities over classes
            output = output - math.log(1. / 48) # subtract the logprob of the class priors (assumed to be uniform)
            likelihood_list.append(output[0])
    likelihoods = torch.transpose(torch.stack(likelihood_list, dim=1), 0, 1).numpy()
    return likelihoods

class MyHMM:
    def __init__(self, state_labels, initial_state_distribution, transition_matrix, eps=1e-200):
        self.eps = eps
        self.pi = np.log(initial_state_distribution + eps)
        self.A = np.log(transition_matrix + eps) #A_{ji} is prob of transitioning from state j to state i
        self.labels = state_labels # a list where self.labels[j] is the index of the phone label belonging to the jth state
        print(self.labels)
        self.N_states = len(self.labels)
        
    def forward(self, state_likelihoods):
        # state_likelihoods.shape is assumed to be (N_timesteps, 48)
        alpha = np.zeros((state_likelihoods.shape[0], self.N_states))

        # Initialization
        alpha[0] = state_likelihoods[0, self.labels] + self.pi[0]

        # Induction
        for i in range(1, state_likelihoods.shape[0]):
            alpha[i] = logsumexp(alpha[i - 1, :, None] + self.A, axis = 0) + state_likelihoods[i, self.labels]

        # Termination
        return alpha[state_likelihoods.shape[0] - 1][self.N_states - 1]
    
    def viterbi(self, state_likelihoods):
        # state_likelihoods.shape is assumed to be (N_timesteps, 48)
        delta = np.zeros((state_likelihoods.shape[0], self.N_states))
        psi = np.zeros((state_likelihoods.shape[0], self.N_states)).astype(np.int32)

        # Initialization
        delta[0] = state_likelihoods[0, self.labels] + self.pi[0] 

        # Induction
        for i in range(1, state_likelihoods.shape[0]):
            delta[i] = np.max(delta[i - 1, :, None] + self.A, axis = 0) + state_likelihoods[i, self.labels]
            psi[i] = np.argmax(delta[i - 1, :, None] + self.A, axis = 0)

        # Backtrace
        S_opt = np.zeros(state_likelihoods.shape[0]).astype(np.int32)
        S_opt[state_likelihoods.shape[0] - 1] = psi[state_likelihoods.shape[0] - 1][self.N_states - 1]
        for n in range(state_likelihoods.shape[0] - 2, -1, -1):
            S_opt[n] = psi[n, int(S_opt[n+1])]

        return delta[state_likelihoods.shape[0]-1, self.N_states-1], S_opt
    
    def viterbi_transition_update(self, state_likelihoods):
        # state_likelihoods.shape is assumed to be (N_timesteps, 48)
        # using Viterbi Decoding function
        delta, S_opt = self.viterbi(state_likelihoods)
        
        new_A = np.zeros_like(self.A, dtype=int)
        # updating A function
        
        for t in range(0, state_likelihoods.shape[0] - 1):
            new_A[S_opt[t]][S_opt[t + 1]] += 1
        new_A = new_A / np.sum(new_A, axis=1)[:, None]
        
        self.A = np.log(new_A + self.eps)
        return

model = MyNet()
model.load_state_dict(torch.load(dir + 'lab3_AM.pt'))

lab3_data = np.load(dir + 'lab3_phone_labels.npz')
phone_labels = list(lab3_data['phone_labels'])
def phones2indices(phones):
    return [phone_labels.index(p) for p in phones]

fee_HMM = MyHMM(phones2indices(['sil', 'f', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
pea_HMM = MyHMM(phones2indices(['sil', 'p', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
rock_HMM = MyHMM(phones2indices(['sil', 'r', 'aa', 'cl', 'k', 'sil']), np.array([0.5,0.5,0,0,0,0]), np.array([[.9,.1,0,0,0,0],[0,.9,.1,0,0,0],[0,0,.9,.1,0,0],[0,0,0,.9,.1,0],[0,0,0,0,.9,.1],[0,0,0,0,0,1]]))
burt_HMM = MyHMM(phones2indices(['sil', 'b', 'er', 'cl', 't', 'sil']), np.array([0.5,0.5,0,0,0,0]), np.array([[.9,.1,0,0,0,0],[0,.9,.1,0,0,0],[0,0,.9,.1,0,0],[0,0,0,.9,.1,0],[0,0,0,0,.9,.1],[0,0,0,0,0,1]]))
see_HMM = MyHMM(phones2indices(['sil', 's', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))
she_HMM = MyHMM(phones2indices(['sil', 'sh', 'iy', 'sil']), np.array([0.5, 0.5, 0, 0]), np.array([[.9,.1,0,0],[0,.9,.1,0],[0,0,.9,.1],[0,0,0,1]]))

# TODO: write your code to use your HMMs below here (or in a new block)

[0, 43, 5, 0]
[0, 10, 5, 0]
[0, 4, 17, 9, 26, 0]
[0, 23, 30, 9, 37, 0]
[0, 1, 5, 0]
[0, 14, 5, 0]


In [ ]:
# computing class likelihoods
def compute_likeli(filename):
    tar_tensor = load_audio_to_melspec_tensor(dir + filename)
    return compute_phone_likelihoods(model, tar_tensor)

def compute_hmm(filename):
    res = []
    likelihood = compute_likeli(filename)
    res.append(fee_HMM.forward(likelihood))
    res.append(pea_HMM.forward(likelihood))
    res.append(rock_HMM.forward(likelihood))
    res.append(burt_HMM.forward(likelihood))
    res.append(see_HMM.forward(likelihood))
    res.append(she_HMM.forward(likelihood))
    res.append(np.argmax(res))
    return res

In [ ]:
result_table = []
result_table.append(compute_hmm('fee.wav'))
result_table.append(compute_hmm('pea.wav'))
result_table.append(compute_hmm('rock.wav'))
result_table.append(compute_hmm('burt.wav'))
result_table.append(compute_hmm('see.wav'))
result_table.append(compute_hmm('she.wav'))
result_table = np.array(result_table)

In [ ]:
import pandas as pd
wav_lists = ['fee','pea','rock','burt','see','she']
hmm_lists = ['fee Hmm','pea Hmm','rock Hmm','burt Hmm','see Hmm','she Hmm']
pd.DataFrame(data = result_table[:,:6], index = wav_lists, columns= hmm_lists)

,fee Hmm,pea Hmm,rock Hmm,burt Hmm,see Hmm,she Hmm
fee,210.104073,179.535837,-92.358515,-85.893900,191.100943,183.554444
pea,250.326733,268.477376,8.474222,71.812346,236.598016,235.196966
rock,-62.373003,-5.380460,157.733017,66.335971,-61.493682,-63.290686
burt,-62.166606,-19.362778,116.139844,220.500684,-74.806470,-96.786347
see,73.563557,75.717636,-176.480611,-158.521863,230.052460,125.818204
she,78.157833,91.857022,-211.203001,-191.500293,124.567268,281.692859


In [60]:
#max word
# print(table[:,6:].flatten())
maxlist = result_table[:,6:].flatten().astype(int)
maxword = [wav_lists[i] for i in maxlist]
table2 = np.zeros((6,8),dtype=object)
table2[:,:6] = result_table[:,:6]
table2[:,6] = maxlist
table2[:,7] = maxword
pd.DataFrame(data = table2, index=wav_lists, columns=['fee Hmm','pea Hmm','rock Hmm','burt Hmm','see Hmm','she Hmm','mostPossibleIndex','mostPossibleWord'])

,fee Hmm,pea Hmm,rock Hmm,burt Hmm,see Hmm,she Hmm,mostPossibleIndex,mostPossibleWord
fee,210.104,179.536,-92.3585,-85.8939,191.101,183.554,0,fee
pea,250.327,268.477,8.47422,71.8123,236.598,235.197,1,pea
rock,-62.373,-5.38046,157.733,66.336,-61.4937,-63.2907,2,rock
burt,-62.1666,-19.3628,116.14,220.501,-74.8065,-96.7863,3,burt
see,73.5636,75.7176,-176.481,-158.522,230.052,125.818,4,see
she,78.1578,91.857,-211.203,-191.5,124.567,281.693,5,she


In [52]:
# Optimal state sequence
# Print the optimal hidden state sequence for the word “rock” using the HMM representing the word “rock”

# paste it into your writeup.
rock_liki = compute_likeli('rock.wav')
rock_optimal_h = rock_HMM.viterbi(rock_liki)
print(rock_optimal_h)

(155.19284689253766, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], dtype=int32))


In [59]:
# Viterbi update
print('The original A matrix:')
print((np.exp(rock_HMM.A)).astype(np.float32))
print(f'The original likelihood: {rock_HMM.forward(rock_liki)}')
rock_HMM.viterbi_transition_update(rock_liki)


print('The updated A matrix:')
print((np.exp(rock_HMM.A)).astype(np.float32))
print(f'The updated likelihood: {rock_HMM.forward(rock_liki)}')

The original A matrix:
[[0.9 0.1 0.  0.  0.  0. ]
 [0.  0.9 0.1 0.  0.  0. ]
 [0.  0.  0.9 0.1 0.  0. ]
 [0.  0.  0.  0.9 0.1 0. ]
 [0.  0.  0.  0.  0.9 0.1]
 [0.  0.  0.  0.  0.  1. ]]
The original likelihood: 157.73301692893546
The updated A matrix:
[[0.9375     0.0625     0.         0.         0.         0.        ]
 [0.         0.9166667  0.08333334 0.         0.         0.        ]
 [0.         0.         0.9285714  0.07142857 0.         0.        ]
 [0.         0.         0.         0.90909094 0.09090909 0.        ]
 [0.         0.         0.         0.         0.8        0.2       ]
 [0.         0.         0.         0.         0.         1.        ]]
The updated likelihood: 158.19678594330463
